In [1]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

In [2]:
# plot 한글출력
plt.rcParams['font.family'] = 'NanumGothic'

In [3]:
# warning message 제거
import warnings
warnings.filterwarnings('ignore')

In [47]:
staffs = pd.read_csv('../01.Data_Collection/staffs.csv')
writer_code = pd.read_csv('../01.Data_Collection/writer_code.csv')
writer_detail = pd.read_csv('../01.Data_Collection/writer_detail.csv')
boxoffice = pd.read_csv('../01.Data_Collection/boxoffice.csv')

print(staffs.shape)
print(writer_code.shape)
print(writer_detail.shape)
print(boxoffice.shape)

(107268, 4)
(1015, 5)
(12215, 6)
(802, 18)


In [5]:
print(staffs.isnull().sum())
staffs[staffs['peopleNm'] == '애덤 비첸']

movieCd        0
movieNm        0
peopleNm       0
staffRoleNm    0
dtype: int64


,movieCd,movieNm,peopleNm,staffRoleNm
15314,20100211,다이노 타임,애덤 비첸,시나리오(각본)


In [12]:
print(writer_code.isnull().sum())
writer_code[writer_code['repRoleNm'].isnull()]
# writer_code[writer_code['peopleNm'] == '애덤 비첸']
# writer_code['repRoleNm'].value_counts()

filmoNames    0
movieNm       0
peopleCd      0
peopleNm      0
repRoleNm     1
dtype: int64


,filmoNames,movieNm,peopleCd,peopleNm,repRoleNm
82,다이노 타임,다이노 타임,10045509,애덤 비첸,NaN


In [14]:
# 애덤 비첸 repRoleNm 채우기

writer_code.loc[writer_code['peopleNm'] == '애덤 비첸', 'repRoleNm'] = '시나리오(각본)'

writer_code.isnull().sum()

filmoNames    0
movieNm       0
peopleCd      0
peopleNm      0
repRoleNm     0
dtype: int64

In [25]:
print(writer_detail.isnull().sum())
writer_detail.sample()
# writer_detail[writer_detail['sex'].isnull()][['peopleCd', 'peopleNm']].unique()
writer_detail[writer_detail['sex'].isnull()]['peopleCd'].unique()

writer_detail[writer_detail['sex'].isnull()]

movieCd          0
movieNm          0
moviePartNm      0
peopleCd         0
peopleNm         0
sex            225
dtype: int64


,movieCd,movieNm,moviePartNm,peopleCd,peopleNm,sex
814,20156807,뽀로로 극장판 컴퓨터 왕국 대모험,시나리오(각본),10057072,이인균,NaN
815,20123681,뽀로로 극장판 슈퍼썰매 대모험,시나리오(각본),10057072,이인균,NaN
904,20100211,다이노 타임,시나리오(각본),10045509,애덤 비첸,NaN
1033,20144442,조선마술사,시나리오(각본),10066948,조정화,NaN
1034,20144442,조선마술사,시나리오(각본),10066948,조정화,NaN
1035,20133811,슬기로운 해법,시나리오(각본),10066948,조정화,NaN
1036,20030063,화성으로 간 사나이,시나리오(각본),10066948,조정화,NaN
1037,20120745,해피버스데이,스토리,10066948,조정화,NaN
1038,20120741,교회누나,시나리오(각본),10066948,조정화,NaN
1039,20020052,묻지마 패밀리,시나리오(각본),10066948,조정화,NaN


In [27]:
writer_detail[writer_detail['peopleCd'] == 10057072]

,movieCd,movieNm,moviePartNm,peopleCd,peopleNm,sex
814,20156807,뽀로로 극장판 컴퓨터 왕국 대모험,시나리오(각본),10057072,이인균,NaN
815,20123681,뽀로로 극장판 슈퍼썰매 대모험,시나리오(각본),10057072,이인균,NaN


In [45]:
writer_code_only = pd.DataFrame(writer_detail['peopleCd'].unique(), columns=['peopleCd'])

writer_code_sex = writer_code_only.merge(writer_detail[['peopleCd', 'peopleNm', 'sex']], how='left', on='peopleCd').drop_duplicates()
writer_code_sex.sample()

,peopleCd,peopleNm,sex
6382,20134495,박정오,남자


In [46]:
writer_code_sex.groupby(['sex'])['peopleNm'].count()

sex
남자    532
여자    205
Name: peopleNm, dtype: int64

각본 공동집필이 많다
- 작품별 한명씩 보고
- 작품별 두명씩 보고

In [49]:
boxoffice.sample()

,순위,영화명,개봉일,매출액,매출액점유율,누적매출액,관객수,누적관객수,스크린수,상영횟수,대표국적,국적,제작사,배급사,등급,장르,감독,배우
674,1,극한직업,2019.1.23,"139,648,179,516",17.6%,"139,648,179,516","16,264,984","16,264,984","2,003","292,577",한국,한국,"(주)어바웃필름,영화사 해그림 주식회사,씨제이이앤엠(주)",씨제이이앤엠(주),15세이상관람가,코미디,이병헌,"류승룡,이하늬,진선규,이동휘,공명,양현민,장진희,허준석,한준우,김명준,나철,정재광"


In [50]:
staffs.sample()

,movieCd,movieNm,peopleNm,staffRoleNm
55193,20156708,석조저택 살인사건,정종길,DI팀


In [56]:
staffs[staffs['staffRoleNm'] == '시나리오(각본)'].groupby(['movieCd'])['peopleNm'].count()

# staffs[(staffs['staffRoleNm'] == '시나리오(각본)') & (staffs['movieCd'] == 20100211)]
staffs[(staffs['staffRoleNm'] == '시나리오(각본)') & (staffs['movieCd'] == 20120422)]

,movieCd,movieNm,peopleNm,staffRoleNm
45679,20120422,거미의 땅,김동령,시나리오(각본)
45680,20120422,거미의 땅,박경태,시나리오(각본)


In [58]:
writer_detail[writer_detail['peopleNm'] == '김동령']
writer_detail[writer_detail['peopleNm'] == '박경태']

,movieCd,movieNm,moviePartNm,peopleCd,peopleNm,sex
4758,20190665,임신한 나무와 도깨비,감독,10028435,박경태,남자
4759,20120422,거미의 땅,감독,10028435,박경태,남자
4760,20120422,거미의 땅,시나리오(각본),10028435,박경태,남자
4761,20120422,거미의 땅,편집,10028435,박경태,남자
4762,20120422,거미의 땅,촬영,10028435,박경태,남자
4763,20090349,사당동 더하기 22,감독,10028435,박경태,남자
4764,20090349,사당동 더하기 22,촬영,10028435,박경태,남자
4765,20060444,나와 부엉이,감독,10028435,박경태,남자


In [61]:
boxoffice[boxoffice['영화명'] == '거미의 땅']

,순위,영화명,개봉일,매출액,매출액점유율,누적매출액,관객수,누적관객수,스크린수,상영횟수,대표국적,국적,제작사,배급사,등급,장르,감독,배우
168,179,거미의 땅,2016.1.14,"6,489,100",0.0%,"7,667,100","1,025","1,291",6,113,한국,한국,(주)시네마달,(주)시네마달,15세이상관람가,다큐멘터리,"박경태,김동령",NaN


각본은 순서를 알 수 없음

감독이 각본을 쓴 경우에는 boxoffice 에서 확인가능